In [28]:
from crewai import Agent, Task, Crew
import requests
import os


In [ ]:
os.environ['OPENAI_API_KEY'] = 'api_key'

# Function to search DuckDuckGo
def duckduckgo_search(query):
    url = f"https://api.duckduckgo.com/?q={query}&format=json&no_redirect=1&no_html=1"
    response = requests.get(url)
    data = response.json()
    return data['RelatedTopics']


In [30]:
class DuckDuckGoAgent(Agent):  # Inherit from Agent
    def __init__(self):
        super().__init__(
            role="DuckDuckGo Searcher",
            goal="Fetch the latest information on {topic} from DuckDuckGo.",
            backstory="You are responsible for conducting a search on DuckDuckGo to find "
                      "relevant information and resources about {topic}. "
                      "Provide the Content Planner with the best available data to inform the content creation process.",
            allow_delegation=False,
            verbose=True
        )

    def fetch_data(self, topic):
        url = f"https://api.duckduckgo.com/?q={topic}&format=json"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            results = data.get("RelatedTopics", [])
            summary = data.get("Abstract", "No summary available.")
            # Collect the top results and the summary
            return summary, [result.get("Text", "No information") for result in results]
        else:
            return "Error fetching data", []

In [31]:
duck_agent = DuckDuckGoAgent()

### Agent itself

In [32]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [33]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [34]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

### Creating Tasks
Define your Tasks, and provide them a description, expected_output and agent.

### Task: Plan

In [35]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [36]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [37]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

### Creating the Crew

In [38]:
# creatng the agents, tasks and verbose (Verbose: Allows us to see how this crew is executing)
# Crew works sequentially So, "ORDER OF TASKS IS IMPORTANT"

crew = Crew(
    agents=[planner, writer, editor, duck_agent],  # Include duck_agent now that it inherits from Agent
    tasks=[plan, write, edit],
    verbose=2
)

### Running the Crew : This is where the TOPIC thing come into play

In [39]:
result = crew.kickoff(inputs={"topic": "What do you know about F1 cars? Also give references in the last paragraph."})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on What do you know about F1 cars? Also give references in the last paragraph..
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer.

Final Answer: 

Content Plan: "What do you know about F1 cars?"

I. Introduction
   A. Brief history of Formula 1 racing and the evolution of F1 cars
   B. Importance of F1 cars in the sport

II. Latest Trends in F1 Cars
   A. Hybrid technology in F1 cars
   B. Aerodynamics and design changes
   C. Safety advancements

III. Key Players
   A. Major manufacturing teams: Mercedes, Red Bull Racing, Ferrari
   B. The role of drivers: Lewis Hamilton, Max Verstappen, Charles Lec

In [41]:
from IPython.display import Markdown
Markdown(result)

# Exploring the World of F1 Cars

## Introduction

Formula 1 racing, often regarded as the apex of motorsport, boasts a rich legacy dating back to its genesis in 1950. The evolution of F1 cars serves as a testament to technological progress and the relentless pursuit of speed and performance. These high-speed machines are the heart of the sport, playing a crucial role in shaping the outcomes of races and championships (1).

## The Current Climate in F1 Cars

In the recent past, the focus of F1 car development has been primarily on hybrid technology. The amalgamation of internal combustion and electric power has not only elevated performance but also initiated a move towards more sustainable racing. Aerodynamics have undergone a comprehensive revamp, with design alterations aimed at enhancing downforce and reducing turbulent air trailing the cars. Additionally, the safety of drivers has leaped forward with the incorporation of features like the halo and improved crash structures (2,3).

## Key Players on the Grid

The F1 grid is predominantly occupied by major manufacturing powerhouses such as Mercedes, Red Bull Racing, and Ferrari. However, their success is not solely a product of their machinery. Drivers such as Lewis Hamilton, Max Verstappen, and Charles Leclerc play an instrumental role in unlocking the potential of these F1 cars. Furthermore, the role of engineers and designers is indispensable, they are the brains behind these high-speed wonders (4).

## Notable Developments

The FIA, the governing body of F1, has recently declared significant changes in F1 car rules and regulations that are slated to be enforced from 2022. The COVID-19 pandemic has inevitably impacted the sport, affecting F1 car development and race schedules. Despite these challenges, the sport is making significant strides towards environmental sustainability, with a goal to achieve net-zero carbon emissions by 2030 (5).

## Wrapping Up

The future of F1 cars appears promising, with the emergence of innovative technologies and an increased focus on sustainability. If you are an aficionado of Formula 1 racing or automobile technology, we encourage you to stay informed with the latest F1 news and races. There's always something new to discover in this rapidly evolving world of F1!

**References**

1. [Official Formula 1 website](www.formula1.com)
2. [F1 Technical](www.f1technical.net)
3. [Autosport](www.autosport.com)
4. [Motorsport](www.motorsport.com)
5. [FIA](www.fia.com)